In [1]:
# Importing Qiskit
# from qiskit import *
# from qiskit.qasm2 import dumps
# from qiskit_aer import Aer

import numpy as np
# import operation as op
from functools import reduce
import random
from random import randrange

In [2]:
def hamming(bits, order):
    '''
    Takes a string of bits to be corrected (bob bits). Bit-wise sums the indices of elements which are '1'. The 0th bit stores the parity
    of the entire block. The location of the error is returned. 
    If the location is not '0', the current 0th parity is matched with that of the parity obtained after flipping the bit at the location obtained. 
    If the parity matches, then the error is found and corrected. If the parity doesn't match then there are more than 1 error.

    If the location is '0', then no error is present.
    
    '''
    
    loc = reduce(lambda x, y : x^y, [i for i, bit in enumerate(bits) if bit == 1])    # x^y will apply xor to the binary rep of i -> index of 1s
    # loc = reduce(op.XOR, [i for i, bit in enumerate(bob_bits) if bit == 1])
    print(f"{loc = }")
    
    binary_rep = f"{bin_rep(loc, order)}"

    par = sum(bits[i] for i in range(0, len(bits)))%2    # Parity of the entire block. It should be 0-Even

    print(f"\n Hamming : ", end = " ")
    if loc != 0 :
        if par != 0 :
            err_count = 1
            print(f"Error found at location : {loc}")

        else :
            err_count = 2
            print("2 errors found")
            
    else : 
        err_count = 0
        print("No errors found")

    print(f" {err_count = }, {loc = }, {binary_rep = }")
    
    return err_count, loc, binary_rep

In [2]:
# Block is the representation of an array consisting the binary data string that has been reshaped as a square matrix
# It has dimensions : dim*dim, where dim = 2**(order/2)

def Order(bits):
    try : return np.ceil(np.log2(len(bits))).astype(int)
    except : return np.ceil(np.log2(bits)).astype(int)
        
def bin_rep(loc, order):
    '''
    Takes a number(int) and order/precision(int) as an input, and returns the binary form with the requested precision.
    '''
    bin_loc = bin(loc)[2:]
    bin_rep = f"{'0'*(order - len(bin_loc))}{bin_loc}"
    
    return bin_rep


def parity(order):
    '''
    Takes in order(int) as a parameter. Returns 2 arrays : 
       - parity_bits : an array containing '0' and the powers of 2 till 2^(order-1)
       - bin_parity : an array of the binary representation of elements of parity_bits   
    '''
    PARITY_DICT = {0:0, **{2**i : 0 for i in range(order)}}    # Initializes the PARITY_DICT
    # parity_bits = np.array([0] + [2**i for i in range(order)]).astype(int)
    bin_parity = np.array([bin_rep(int(i), int(order)) for i in PARITY_DICT.keys()])

    return PARITY_DICT, bin_parity
    
    
def parity_locs(order):
    '''
    Takes in order(int) as a parameter. Returns an array :
        - parity_locs : A block(array reshaped as square matrix) with 1 at the locations of parity bits
    '''
    parity_locs = np.full(2**order, '-', dtype = object)
    PARITY_DICT = parity(order)[0]
    
    for loc in PARITY_DICT.keys() : parity_locs[loc] = '1'

    return parity_locs


def block(bits, order):
    dim = int(2**(order/2))
    # bits = bits.astype(int)
    # print(f" {len(bits) = } ")
    # print(f'{type(bits[0])= }')
    
    if len(bits) != 2**order : return f"key size (= {len(bits)}) not an exponent of 2 : {bits}"
        
    elif not order%2 : 
        try :
            return(f"block : \n {bits.reshape(dim, dim)} \n Shape of the block : {dim}*{dim}")
        except : 
            int_bits = np.array([int(bit) for bit in bits])
            return(f"block : \n {int_bits.reshape(dim, dim)} \n Shape of the block : {dim}*{dim}")

    else :
        return(f"bit string(Order is odd, can't project to a block) : \n")#{bits} \n Shape of the block : {bits.shape}")

In [ ]:
def create_parity_block(bits, order, PARITY_DICT):
    '''
    A function to take a string of bits shared through QKD, and to morph them into a parity block with parity bits(unchecked) embedded
    '''
    block = np.zeros(2**order).astype('uint8')
    ### Encode alice_keys and get the PARITY_DICT before proceeding to bob_key
    
    j = 0 
    for i in range(2**order) : 
        if i in PARITY_DICT.keys() : block[i] = PARITY_DICT[i]
        elif j < len(bits) :    
            block[i] = bits[j]
            j += 1

    block, PARITY_DICT = encode_parity(block, order, PARITY_DICT)
    
    return block

In [5]:
def encode_parity(bits, order, PARITY_DICT):
    # order = np.ceil(np.log2(len(alice_bits)))
    
    sub_block = int(2**(order - 1))
    parity_of = np.zeros((len(PARITY_DICT), sub_block)).astype(int)   # An array to store the locations affecting the parity p
    
    for p in range(1, order+1) :    # checking for 1 at position p. eg : bin(45) = 101101
    
        bit_index = 2**(p-1)
        highlight = np.zeros(2**order).astype(int)                        # Highlights the locations affected by the current parity bit
        # print(f"bin rep of {bit_index = } : {bin_parity[p]}")
        Sum = 0
    
        for i in range(sub_block):                                         #  Order-1 = 5. range(5) = 0, 1, 2, 3, 4 => order-2
            bin_index = bin_rep(i, order-1)                                # Index(in binary formin binary form) for the data bits : 5 digits : 00010
            bin_index = bin_index[: order-p] + '1' + bin_index[order-p :]
            index = int(bin_index, base = 2)                                # Gives the index(int) of the elements to be considered for the current parity element
            
            parity_of[p, i] = index
            # highlight[index] = 1

            if bit_index != index :
                Sum = np.mod(Sum + bits[index], 2)

        # PARITY_DICT[bit_index] = np.mod( sum( bits[parity_of[p, i]] for i in range(sub_block) if bit_index != parity_of[p, i] ), 2 )

        PARITY_DICT[bit_index]= Sum
        bits[bit_index] = PARITY_DICT[bit_index]
            
        # print(highlight.reshape(dim, dim))
    
    PARITY_DICT[0] = sum( bits[i] for i in range(1, 2**order) )%2
    bits[0] = PARITY_DICT[0]
    # print(f"Parity locations : \n{parity_of[1:]}") 
    
    print("\n Hamming Results : ", hamming(bits, order))
    print(f" Uncorrected {block(bits, order)}")
    
    return bits, PARITY_DICT